In [3]:
import os
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
%cd /content/drive/MyDrive/컴퓨터 비전/computer-vision
import os
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from dataloader.dataset import CustomDataset
from model.unet import UNet
from tool.encode import rle_encode
from tool.transform import fisheye
import albumentations as A
from PIL import Image
from albumentations.pytorch import ToTensorV2

SAVE_ROOT = "/content/drive/MyDrive/컴퓨터 비전/computer-vision/model/checkpoint"
DATA_ROOT = "/content/drive/MyDrive/컴퓨터 비전/data"

def set_dataloader(csv_file, valid = False, batch_size = 16):
    def set_tta():
        transform = A.Compose(
            [
              fisheye([-1, 3.5, 0, 0]),
              A.CenterCrop(600, 930),
              A.Resize(224, 224),
              A.Normalize(),
              ToTensorV2()
            ])
        return transform

    def set_train_aug():
        transform = A.Compose(
            [
              fisheye([-1, 3.5, 0, 0]),
              A.CenterCrop(600, 930),
              A.Resize(224, 224),
              A.Normalize(),
              ToTensorV2()
            ])
        return transform

    if not valid: t = set_train_aug()
    else: t = set_tta()

    dataset = CustomDataset(data_root = DATA_ROOT, csv_file=csv_file, transform= t, infer=False)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    return dataloader

def set_model():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = UNet().to(device)
    return model, device

def train():
    model, device = set_model()
    train_dataloader = set_dataloader('train_source.csv')
    valid_dataloader = set_dataloader('val_source.csv')
    model_name = "unet"
    val_every = 5
    max_epoch = 20

    # loss function과 optimizer 정의
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # Training loop
    min_loss = 1e9
    for epoch in range(max_epoch):
        model.train()
        epoch_loss = 0
        for images, masks in tqdm(train_dataloader):
            images = images.float().to(device)
            masks = masks.long().to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, masks.squeeze(1))
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        train_loss = epoch_loss/len(train_dataloader)
        print(f'Epoch {epoch+1}, Train Loss: {train_loss}')

        # Validation loop
        if epoch==0 or (epoch+1)%val_every==0:
            model.eval()
            with torch.no_grad():
                vloss = 0
                for images, masks in tqdm(valid_dataloader):
                    images = images.float().to(device)
                    masks = masks.long().to(device)

                    outputs = model(images)
                    loss = criterion(outputs, masks.squeeze(1))
                    vloss += loss.item()
                val_loss = vloss / len(valid_dataloader)
                print(f'Valid Loss: {val_loss}')
                if min_loss > val_loss:
                    print(f"Minimum Loss!! let's Saved... (Check {model_name}+_epoch{epoch+1}.pt)")
                    min_loss = val_loss
                    torch.save(model, os.path.join(SAVE_ROOT, model_name+f"_epoch{epoch+1}.pt"))
train()

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


/content/drive/MyDrive/컴퓨터 비전/computer-vision


  0%|          | 0/138 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
 99%|█████████▉| 137/138 [06:38<00:01,  1.38s/it]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|██████████| 138/138 [06:38<00:00,  2.89s/it]


Epoch 1, Train Loss: 1.2635638839092806


100%|██████████| 30/30 [02:33<00:00,  5.10s/it]


...

100%|██████████| 138/138 [06:52<00:00,  2.99s/it]


Epoch 15, Train Loss: 0.20784138456202936


100%|██████████| 30/30 [01:26<00:00,  2.87s/it]


Valid Loss: 0.48820096453030903
minimum loss!! saved...


 67%|██████▋   | 92/138 [04:39<01:24,  1.84s/it]